In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

## Download data

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2024-05-29 19:16:22--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.152, 18.239.238.133, 18.239.238.119, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   137MB/s    in 0.3s    

2024-05-29 19:16:23 (137 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-29 19:16:34--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.152, 18.239.238.212, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet’

yellow_tripdata_202 100%[===================>]  45.54M   113MB/s    in 0.4s    

2024-05-29 19:16:34 (113 MB/s) - ‘yellow_tripdata_2023-02.parquet’ saved [47748012/47748012]



## EDA

In [2]:
df = pd.read_parquet("yellow_tripdata_2023-01.parquet")

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df.shape

(3066766, 19)

In [5]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")

There are 3066766 rows and 19 columns


In [6]:
df["duration"] = df["tpep_dropoff_datetime"]-df["tpep_pickup_datetime"]

In [7]:
df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)

In [8]:
duration_std = df.duration.std()
print(f"The standard deviation of the trips duration in January is {duration_std:.2f}")

The standard deviation of the trips duration in January is 42.59


In [9]:
rows, cols = df.shape

In [10]:
df = df[(df["duration"] <= 60) & (df["duration"] >= 1)]

In [11]:
new_rows, _ = df.shape

In [12]:
print(f"There is {new_rows/rows * 100:.2f}% data left")

There is 98.12% data left


In [13]:
features = ["PULocationID", "DOLocationID"]

In [14]:
df[features].dtypes

PULocationID    int64
DOLocationID    int64
dtype: object

In [15]:
df[features] = df[features].astype(str)

In [16]:
df[features].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [17]:
train_dict = df[features].to_dict(orient="records")

In [18]:
dv = DictVectorizer()

In [19]:
X_train = dv.fit_transform(train_dict)

In [20]:
dim_features = len(dv.feature_names_)
print(f"There are {dim_features} columns")

There are 515 columns


In [21]:
y_train = df["duration"].values

In [22]:
lr = LinearRegression()

In [23]:
lr.fit(X_train, y_train)

LinearRegression()

In [24]:
y_pred = lr.predict(X_train)

In [25]:
rmse = root_mean_squared_error(y_train, y_pred)

In [26]:
print(f"The RMSE on train is {rmse}")

The RMSE on train is 7.649262443101424


### Validation

In [27]:
df_valid = pd.read_parquet("yellow_tripdata_2023-02.parquet")

In [28]:
df_valid["duration"] = df_valid["tpep_dropoff_datetime"]-df_valid["tpep_pickup_datetime"]

In [29]:
df_valid.duration = df_valid.duration.apply(lambda x: x.total_seconds() / 60)

In [30]:
df_valid = df_valid[(df_valid["duration"] <= 60) & (df_valid["duration"] >= 1)]

In [31]:
df_valid[features] = df_valid[features].astype(str)

In [32]:
valid_dict = df_valid[features].to_dict(orient="records")

In [33]:
X_valid = dv.transform(valid_dict)

In [34]:
y_valid = df_valid["duration"].values

In [35]:
y_pred = lr.predict(X_valid)

In [39]:
rmse = root_mean_squared_error(y_valid, y_pred)

In [40]:
print(f"The RMSE on validation is {rmse}")

The RMSE on validation is 13.328414086125777
